In [2]:
import random
import operator
from deap import creator, gp, base, tools, algorithms

In [193]:
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
    
    def __str__(self):
        v = self.value
        if v == 14:
            v = 'A'
        elif v == 11:
            v = 'J'
        elif v == 12:
            v = 'Q'
        elif v == 13:
            v = 'K'
        return str(v) + self.suit

class Deck:
    def __init__(self, r=4):
        self.cards = []
        self.draw_queue = []
        self.random_num = r
    
    def __str__(self):
        return_value = ""
        for card in self.draw_queue:
            return_value += str(card) + '\n'
        
        return return_value
    
    def setup(self, n=1):
        if len(self.cards) == 0:
            for i in range(0, n):
                for v in range(2, 15):
                    for s in ['S', 'D', 'C', 'H']:
                        self.cards.append(Card(v, s))

        self.draw_queue = []
        self.shuffle(self.random_num)
        self.random_num += 1
    
    def shuffle(self, r=4):
        self.draw_queue = list(self.cards)
        #for i in range(0, 10):
        #    print self.draw_queue[i]
        #random.Random(self.random_num).shuffle(self.draw_queue)
        random.shuffle(self.draw_queue)

    def draw(self):
        return self.draw_queue.pop()

In [4]:
class Player:
    def __init__(self, hand, pot):
        self.hand = hand
        self.pot = pot
        self.folded = False
        self.current_bet = 0
        self.last_move = None
    
    def __str__(self):
        result = ''
        result += 'Hand: ' + str(self.hand[0]) + '  ' + str(self.hand[1])
        return result + '\nPot: ' + str(self.pot) + '\n'  
    
    def softReset(self):
        self.folded = False
        self.current_bet = 0
        self.last_move = None        
    
    def bid(self, amount):
        bid_value = amount
        if amount > self.pot:
            bid_value = self.pot
        
        self.pot = self.pot - bid_value
        self.current_bet += bid_value
        
        return bid_value

In [62]:
class Gameloop:
    def __init__(self, number_of_players, starting_player_pot, small_blind):
        self.number_of_players = number_of_players
        self.players = []
        for n in range(0, self.number_of_players):
            self.players.append(Player(0, starting_player_pot))
        self.current_player = 0
        self.button = 0
        self.small_blind = small_blind
        self.big_blind = 2 * small_blind
        self.deck = Deck()
        self.pot = 0
        self.board = []
        self.active_players = set()
        self.highest_bidder = None
        self.current_bid = 0
    
    def __str__(self):
        result = ''
        for player in self.players:
            result += str(player)
        
        return result
    
    def setup(self):
        self.deck.setup()
        self.pot = 0
        self.button = (self.button + 1) % self.number_of_players
        self.board = []
        
        small_blind_index = (self.button + 1) % self.number_of_players
        big_blind_index = (self.button + 2) % self.number_of_players
        
        #print str(small_blind_index) + ", " + str(big_blind_index)
        
        for i in range(0, self.number_of_players):
            hand = (self.deck.draw(), self.deck.draw())
            self.players[i].softReset()
            self.players[i].hand = hand
            if i == small_blind_index:
                self.pot += self.players[i].bid(self.small_blind)
            elif i == big_blind_index:
                self.pot += self.players[i].bid(self.big_blind)
        
        self.current_player = ((self.button + 3) % self.number_of_players)
        self.active_players = set(range(0, self.number_of_players))
        self.highest_bidder = big_blind_index
        self.current_bid = self.big_blind
        
        #self.button = (self.button + 1) % self.number_of_players
    
    def flop(self):
        self.board.append(self.deck.draw())
        self.board.append(self.deck.draw())
        self.board.append(self.deck.draw())
    
    def turn(self):
        self.board.append(self.deck.draw())
    
    def river(self):
        self.board.append(self.deck.draw())
    
    '''
    value  :   hand
    10 : Royal Straight Flush
    9  : Straight Flush
    8  : Four of a Kind
    7  : Full House
    6  : Flush
    5  : Straight
    4  : Three of a Kind
    3  : Two Pair
    2  : One Pair
    0  : High Card
    '''
    
    def flush(self, order_by_suit):
        max_value = 0
        max_suit = None
        for suit in order_by_suit:
            length = len(order_by_suit[suit])
            if length >= 5:
                max_value = length
                max_suit = suit
                break
        
        if max_value < 5:
            return (0, [])

        cards = order_by_suit[max_suit]
        
        if len(set([14, 13, 12, 11, 10]) & set(cards)) == 5: # Royal Straight Flush
            return (10, [])
        
        if 14 in cards:
            cards = [1] + cards # To guarantee that A's can also sequel with low card sequences
        difference_list = [cards[i+1]-cards[i] for i in range(0, len(cards)-1)]
        string_difference_list = ''.join([str(x) for x in difference_list])
        if '1111' in string_difference_list: # Straight Flush
            index = string_difference_list.find('1111')
            
            return (9, [cards[index+4]])
        
        return (6, [cards[-1], cards[-2], cards[-3], cards[-4], cards[-5]]) # Flush

    def xOfAKind(self, order_by_value, cards):
        values = order_by_value.values()
        keys = order_by_value.keys()
        card_set = set(cards)
        if 4 in values:
            card_value = keys[values.index(4)] 
            temp = card_set.difference(set([card_value]))
            return (8, [card_value, max(temp)]) # 4 of a Kind
        
        if (3 in values and 2 in values) or (3 in values and values.count(3) > 1):
            dict_of_keys = {2: [], 3: []}
            for key in order_by_value:
                if order_by_value[key] == 3:
                    dict_of_keys[3].append(key)
                    dict_of_keys[2].append(key)
                elif order_by_value[key] == 2:
                    dict_of_keys[2].append(key)
            
            max_3_card = max(dict_of_keys[3])
            max_2_card = max(set(dict_of_keys[2]).difference(set([max_3_card])))
            
            return (7, [max_3_card, max_2_card]) # Full House
        
        if 3 in values:
            dict_of_keys = {3: []}
            for key in order_by_value:
                if order_by_value[key] == 3:
                    dict_of_keys[3].append(key)
            
            card_value = max(dict_of_keys[3])
            max_1st_card_left = max(card_set.difference(set([card_value])))
            max_2nd_card_left = max(card_set.difference(set([card_value, max_1st_card_left])))
            
            return (4, [card_value, max_1st_card_left, max_2nd_card_left]) # 3 of a Kind
        
        if 2 in values:
            list_of_pair_keys = []
            for key in order_by_value:
                if order_by_value[key] == 2:
                    list_of_pair_keys.append(key)

            rank = 2
            if len(list_of_pair_keys) > 1:
                rank = 3
            
            max_1st_card = max(list_of_pair_keys)
            set_of_pair_keys = set(list_of_pair_keys).difference(set([max_1st_card]))
            max_2nd_card = 0
            if len(set_of_pair_keys) > 0:
                max_2nd_card = max(set_of_pair_keys)
            
            temp = list(card_set.difference(set([max_1st_card, max_2nd_card])))
            temp = sorted(temp)
            
            if rank == 3:
                return (3, [max_1st_card, max_2nd_card, temp[-1]])
            
            return (2, [max_1st_card, temp[-1], temp[-2], temp[-3], temp[-4]])
            #return 3 if values.count(2) >= 2 else 2 # Two or One Pair

        return (0, [])
    
    def straight(self, cards):
        if 14 in cards:
            cards = [1] + cards # To guarantee that A's can also sequel with low card sequences

        difference_list = [cards[i+1].value-cards[i].value for i in range(0, len(cards)-1)]
        string_difference_list = ''.join([str(x) for x in difference_list])
        if '1111' in string_difference_list: # Straight
            index = string_difference_list.find('1111')
            return (5, [cards[index+4].value])
    
        return (0, [])
    
    def rankHand(self, player_index):
        player = self.players[player_index]
        
        list_of_all_cards = self.board + [player.hand[0], player.hand[1]]
        list_of_all_cards.sort(key=operator.attrgetter('value'))
        
        order_by_suit = {'S': [], 'H': [], 'C': [], 'D': []}
        for card in list_of_all_cards:
            order_by_suit[card.suit].append(card.value)
        
        order_by_value = {}
        for card in list_of_all_cards:
            if card.value not in order_by_value:
                order_by_value[card.value] = 1
            else:
                order_by_value[card.value] += 1
        
        hand_rank = (0, [])
        
        hand_rank = self.flush(order_by_suit)
        if hand_rank[0] < 9:
            temp = self.xOfAKind(order_by_value, [card.value for card in list_of_all_cards])
            hand_rank = temp if temp > hand_rank else hand_rank
        if hand_rank[0] < 5:
            temp = self.straight(list_of_all_cards)
            hand_rank = temp if temp > hand_rank else hand_rank
        
        if hand_rank[0] == 0:
            temp = [x.value for x in player.hand]
            temp = sorted(temp)
            
            hand_rank = (0, [temp[-1], temp[-2]])
        
        return hand_rank
    
    def isBettingRoundOver(self):
        if len(self.active_players) <= 1:
            #print "1 player"
            return True
        
        if self.current_player == self.highest_bidder and self.players[self.current_player].last_move != None:
            #print "All checks"
            return True
        
        return False

    def nextPlayer(self):
        self.current_player = (self.current_player + 1) % self.number_of_players
        while self.current_player not in self.active_players:
            self.current_player = (self.current_player + 1) % self.number_of_players
    
    def raiseBet(self):
        player = self.players[self.current_player]
        
        amount = self.current_bid - player.current_bet + self.big_blind
        value_of_bid = player.bid(amount)
        self.pot += value_of_bid
        self.current_bid += self.big_blind
        self.highest_bidder = self.current_player

        player.last_move = 'raise'
        
        self.nextPlayer()
    
    def fold(self):
        player = self.players[self.current_player]
        self.active_players.remove(self.current_player)
        
        player.last_move = 'fold'
        
        self.nextPlayer()
    
    def check(self):
        player = self.players[self.current_player]
        
        amount = self.current_bid - player.current_bet
        
        player.last_move = 'check'

        if amount > 0:
            value_of_bid = player.bid(amount)
            self.pot += value_of_bid
            self.nextPlayer()
    
    def possible_moves(self):
        moves = ['raise', 'check', 'fold']
        
        #player = self.players[self.current_player]
        
        #if player.current_bet == self.current_bid:
        #    moves.append('check')
        
        return moves
    
    def executeMove(self, move):
        if move == 'raise':
            self.raiseBet()
        
        elif move == 'check':
            self.check()
        
        elif move == 'fold':
            self.fold()
    
    def isHandHigher(self, hand1, hand2):
        if hand1[0] > hand2[0]:
            return 0 # CHALLENGING HAND IS LOWER
        
        if hand2[0] > hand1[0]:
            return 1 # CHALLENGING HAND IS HIGHER
        
        for i in range(0, len(hand1[1])):
            if hand1[1][i] > hand2[1][i]:
                return 0 # CHALLENGING HAND IS LOWER
            if hand2[1][i] > hand1[1][i]:
                return 1 # CHALLENGING HAND IS HIGHER
        
        return 2 # TIE
    
    def winner(self):
        results = {}
        for player_index in self.active_players:
            results[player_index] = self.rankHand(player_index)
        
        #print results
        
        highest_hand = None
        winning_players = []
        
        for key in results:
            if highest_hand == None:
                highest_hand = results[key]
                winning_players.append(key)
            else:
                evaluation = self.isHandHigher(highest_hand, results[key])
                if evaluation == 1:
                    highest_hand = results[key]
                    winning_players = [key]
                elif evaluation == 2: 
                    winning_players.append(key)
        
        return winning_players
    

In [198]:
class GameHandler:
    def __init__(self, number_of_players, agents, starting_player_pot, small_blind):
        self.starting_player_pot = starting_player_pot
        self.game = Gameloop(number_of_players, starting_player_pot, small_blind)
        self.agents = agents
    
    def gameloop(self):
        for i in range(0, self.game.number_of_players):
            self.game.players[i].pot = self.starting_player_pot
            
        self.game.setup()
        
        for i in range(0, self.game.number_of_players):
            self.agents[i].setHand(self.game.players[i].hand)
            self.agents[i].generateOffset()
            #self.game.players[i].pot = self.starting_player_pot
        
        counter = 0
        while not self.game.isBettingRoundOver():
            current_agent = self.game.current_player
            
            move = self.agents[current_agent].decide(self.game, self.game.possible_moves())
            #print(str(current_agent) + ": " + str(move))
            self.game.executeMove(move)
            
            #counter += 1
            #if counter == 100:
            #    break
        
        self.game.flop()
        self.game.turn()
        self.game.river()
        
    
    def run(self):
        self.gameloop()
        
        winners = self.game.winner()
        #print winners
        num_of_winners = float(len(winners))
        for win_player_index in winners:
            self.game.players[win_player_index].pot += int(float(self.game.pot) / num_of_winners)        
        
        return winners, self.game.players[0].pot

    def runGetAllPlayerPots(self):
        self.gameloop()
        
        winners = self.game.winner()
        num_of_winners = float(len(winners))
        for win_player_index in winners:
            self.game.players[win_player_index].pot += int(float(self.game.pot) / num_of_winners)        
        
        return winners, [self.game.players[i].pot for i in range(self.game.number_of_players)]


In [159]:
#### WIKIPEDIA ALGORITHM AI

import math

class TierHandAI:
    def __init__(self):
        self.offset = 0.0
    
    def setHand(self, hand):
        self.hand = [hand[0].value, hand[1].value]
        self.isSameSuit = True if hand[0].suit == hand[1].suit else False
        self.hand_value = self.evaluateHand()
    
    def generateOffset(self):
        self.offset = random.randint(0, 100)
        self.fold_offset = random.uniform(-0.2, 0.2)
    
    def evaluateHand(self):
        hand_value = 0
        highest_card = max(self.hand)
        smallest_card = min(self.hand)
        
        if highest_card == 14:
            hand_value += 10.0
        elif highest_card == 13:
            hand_value += 8.0
        elif highest_card == 12:
            hand_value += 7.0
        elif highest_card == 11:
            hand_value += 6.0
        else:
            hand_value += float(highest_card) / 2.0
        
        if self.hand[0] == self.hand[1]:
            hand_value *= 2.0
            if self.hand[0] == 5:
                hand_value += 1.0
        
        if self.isSameSuit:
            hand_value += 2.0

        point_gap = highest_card - smallest_card
        if highest_card == 14 and smallest_card < 6:
            point_gap = smallest_card - 1

        if point_gap == 2:
            hand_value -= 1.0
        elif point_gap == 3:
            hand_value -= 2.0
        elif point_gap == 4:
            hand_value -= 4.0
        elif point_gap > 4:
            hand_value -= 5.0
        
        if point_gap < 2 and highest_card < 12:
            hand_value += 1.0
        
        if hand_value < 0:
            hand_value = 0.0

        return hand_value
        
    def decide(self, gamestate, available_actions):
        self_current_bet = gamestate.players[gamestate.current_player].current_bet
        buy_in = gamestate.current_bid - self_current_bet
        
        self_current_bet = float(self_current_bet) / float(gamestate.big_blind)
        
        if self.hand_value == 0.0:
            if buy_in <= 1:
                return 'check'
            return 'fold'
        
        MAX_BID_LEVEL = 20.0
        
        random_value = random.uniform(0, 1)
        current_total_bid_level = float(gamestate.current_bid) / float(gamestate.big_blind)
        #print(current_total_bid_level)
        
        normalized_hand_value = self.hand_value / 20.0
        
        #raise_chance = normalized_hand_value
        highest_bid_level = MAX_BID_LEVEL * normalized_hand_value + self.offset
        #print(highest_bid_level)
        #fold_chance = (1.0 - normalized_hand_value) / 2.0
        fold_chance = (1.0 - math.log(self.hand_value, 20)) + self.fold_offset#+ self.offset
        
        max_buy_in = gamestate.number_of_players * normalized_hand_value
        
        if buy_in > max_buy_in:
            return 'fold'
        
        if random_value < fold_chance:
            if buy_in == 0 or self_current_bet > 3:
                return 'check'
            return 'fold'
        
        if highest_bid_level > current_total_bid_level:
            return 'raise'
        
        return 'check'
        
        #level_of_bid = float(gamestate.current_bid - gamestate.players[gamestate.current_player].current_bet)
        #level_of_bid = self.level_of_bid / float(gamestate.big_blind)
        
        # Hand Value interval = [0.0, 20.0]        

In [164]:
### Play Poker Like the Pros (Book), Author: Phil Hellmuth

class TopTenHandAI:
    def __init__(self):
        self.offset = 0.0
    
    def setHand(self, hand):
        self.hand = [hand[0].value, hand[1].value]
        self.isSameSuit = True if hand[0].suit == hand[1].suit else False
        self.hand_value = self.evaluateHand()
        self.tier = 0
    
    def generateOffset(self):
        self.offset = random.randint(0, 100)
    
    def evaluateHand(self):
        hand_value = 0.0
        card1 = max(self.hand)
        card2 = min(self.hand)
        
        if card1 == card2 and card1 >= 7:
            hand_value = 1.0
        else:
            if card2 >= 12:
                hand_value = 1.0
        
        if hand_value > 0.0:
            self.tier = 1
            if card1 <= 8:
                self.tier = 3
            elif card1 != card2 and card2 == 12:
                self.tier = 3
            elif card1 == card2 and card1 <= 11:
                self.tier = 2

        return hand_value
        
    def decide(self, gamestate, available_actions):
        buy_in = gamestate.current_bid - gamestate.players[gamestate.current_player].current_bet
        
        if self.hand_value == 0.0:
            if buy_in <= 1:
                return 'check'
            return 'fold'
        
        MAX_BID_LEVEL = 20.0
        
        current_total_bid_level = float(gamestate.current_bid) / float(gamestate.big_blind)

        highest_bid_level = MAX_BID_LEVEL + self.offset
        max_buy_in = gamestate.number_of_players * float(self.tier) / 3.0
        
        if highest_bid_level > current_total_bid_level:
            if buy_in > max_buy_in:
                return 'fold'
            
            return 'raise'
        
        return 'check'
        

In [9]:
class AlwaysRaiseAI:
    def __init__(self):
        pass
    
    def setHand(self, hand):
        pass
    
    def decide(self, gamestate, available_actions):
        return 'raise'

In [10]:
from deap import creator, gp, base, tools, algorithms

import operator
import random as randomizer

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMax)
creator.create("GameHeuristicIndividualMax", gp.PrimitiveTree, fitness=creator.FitnessMax)

def IfThenElse(input_condition, output1, output2):
    if input_condition:
        return output1
    else:
        return output2

def isSameSuit(hand):
    if hand[0].suit == hand[1].suit:
        return True
    
    return False

def hasDoubles(hand):
    if hand[0].value == hand[1].value:
        return True
    
    return False

def hasCard(hand, value):
    if hand[0].value == value or hand[1].value == value:
        return True
    
    return False

def highestCard(hand):
    temp = [hand[0].value, hand[1].value]
    
    return max(temp)

def highestCardGE(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card >= value:
        return True
    return False

def highestCardEQ(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card == value:
        return True
    return False

def highestCardLE(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card <= value:
        return True
    return False

def lowestCard(hand):
    temp = [hand[0].value, hand[1].value]
    
    return min(temp)

def lowestCardGE(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card >= value:
        return True
    return False

def lowestCardEQ(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card == value:
        return True
    return False

def lowestCardLE(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card <= value:
        return True
    return False

def totalPotGE(potSize, value):
    if isinstance(potSize, list):
        return True
    return potSize >= value

def totalPotLE(potSize, value):
    if isinstance(potSize, list):
        return True
    return potSize <= value

def eq(val1, val2):
    return operator.eq(val1, val2)

def ge(val1, val2):
    return operator.ge(val1, val2)

def le(val1, val2):
    return operator.le(val1, val2)

pset = gp.PrimitiveSetTyped("MoveSelector", [list, int], str)
#pset.renameArguments(ARG0="Hand", ARG1="PotSize")

pset.addTerminal(True, bool)
pset.addTerminal(False, bool)
for i in range(2,14):
    pset.addTerminal(i, int)
pset.addTerminal("raise", str)
pset.addTerminal("check", str)
pset.addTerminal("fold", str)
pset.addPrimitive(operator.and_, [bool, bool], bool)
#pset.addPrimitive(eq, [int, int], bool)
#pset.addPrimitive(ge, [int, int], bool)
#pset.addPrimitive(le, [int, int], bool)

pset.addPrimitive(IfThenElse, [bool, str, str], str)
pset.addPrimitive(isSameSuit, [list], bool)
pset.addPrimitive(hasDoubles, [list], bool)
pset.addPrimitive(hasCard, [list, int], bool)
pset.addPrimitive(highestCardGE, [list, int], bool)
#pset.addPrimitive(highestCardEQ, [list, int], bool)
pset.addPrimitive(highestCardLE, [list, int], bool)
pset.addPrimitive(lowestCardGE, [list, int], bool)
#pset.addPrimitive(lowestCardEQ, [list, int], bool)
pset.addPrimitive(lowestCardLE, [list, int], bool)
pset.addPrimitive(totalPotGE, [int, int], bool)
pset.addPrimitive(totalPotLE, [int, int], bool)


In [11]:
possible_strings = [
                "raise",
                "check",
                "fold"
               ]

possible_ints = range(2, 15)
possible_pot_ints = range(1, 30)
#possible_bools_operations = ["and_", "eq", "ge", "le", "isSameSuit", "hasDoubles", "hasCard"]
#possible_bools_operations = ["and_", "isSameSuit", "hasDoubles", "hasCard", "highestCardGE", "highestCardEQ", "highestCardLE", "lowestCardGE", "lowestCardEQ", "lowestCardLE"]
possible_bools_operations = ["and_", "isSameSuit", "hasDoubles", "highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]
possible_bools_operations.extend(["totalPotGE", "totalPotLE"])


In [211]:
import random
import multiprocessing

def generateBoolOp():
    operation = random.choice(possible_bools_operations)
    
    if operation == "and_":
        param1 = generateBoolOp()
        param2 = generateBoolOp()
    else:
        #if len(operation) < 5:
        if "totalPot" in operation:
            param1 = "ARG1"
            #param2 = str(random.choice(possible_ints))
            param2 = str(random.choice(possible_pot_ints))
        else:
            param1 = "ARG0"
            if operation != "hasDoubles" and operation != 'isSameSuit':
                param2 = str(random.choice(possible_ints))
            else:
                return operation + "(" + param1 + ")"
            
    
    return operation + "(" + param1 + "," + param2 + ")"

def generateTerminal():
    term = random.choice(possible_strings)

    return term #+ "(Moves," + param1 + "," + param2 + ")"
    
def generateIndividualString(depth):
    if depth == 0:
        return generateTerminal()
    else:
        result = ""
        for i in range(0, depth):
            result += "IfThenElse(" + generateBoolOp() + "," + generateTerminal() + ","
        
        result += generateTerminal()
        for i in range(0, depth):
            result += ")"
        
        return result

def generateIndividual(depth, pset, type_=None):
    return gp.PrimitiveTree.from_string(generateIndividualString(depth), pset)

def parseAndCondition(condition):
    result = []
    i = 1
    while i < len(condition):
        if condition[i].name == "isSameSuit":
            result.append([isSameSuit])
            i += 1
        elif condition[i].name == "hasDoubles":
            result.append([hasDoubles])
            i += 1
        elif condition[i].name == "totalPotGE":
            result.append([totalPotGE, int(condition[i+2].name)])
        elif condition[i].name == "totalPotLE":
            result.append([totalPotLE, int(condition[i+2].name)])
        elif condition[i].name == "highestCardLE":
            result.append([highestCardLE, int(condition[i+2].name)])
        elif condition[i].name == "highestCardGE":
            result.append([highestCardGE, int(condition[i+2].name)])
        elif condition[i].name == "lowestCardLE":
            result.append([lowestCardLE, int(condition[i+2].name)])
        elif condition[i].name == "lowestCardGE":
            result.append([lowestCardGE, int(condition[i+2].name)])
        i += 1
    
    return result

def parseCondition(condition):
    #print [x.name for x in condition]
    result = []
    
    if condition[0].name == "isSameSuit":
        result = [isSameSuit]
    elif condition[0].name == "hasDoubles":
        result = [hasDoubles]
    elif condition[0].name == "totalPotGE":
        result = [totalPotGE, int(condition[-1].name)]
    elif condition[0].name == "totalPotLE":
        result = [totalPotLE, int(condition[-1].name)]
    elif condition[0].name == "highestCardLE":
        result = [highestCardLE, int(condition[-1].name)]
    elif condition[0].name == "highestCardGE":
        result = [highestCardGE, int(condition[-1].name)]
    elif condition[0].name == "lowestCardLE":
        result = [lowestCardLE, int(condition[-1].name)]
    elif condition[0].name == "lowestCardGE":
        result = [lowestCardGE, int(condition[-1].name)]
    elif condition[0].name == "and_":
        result = parseAndCondition(condition)
    
    return result

def parseAllConditions(conditions):
    result = []
    for condition in conditions:
        result.append(parseCondition(condition))
    
    return result

def handsThatMeetCondition(condition, hands):
    result = []
    
    if len(condition) == 1:
        for hand in hands:
            if condition[0](hand):
                result.append(hand)
    elif len(condition) == 2 and not isinstance(condition[0], list):
        for hand in hands:
            if condition[0](hand, condition[1]):
                result.append(hand)
    else:
        compiled_conditions = condition
        #print compiled_conditions
        for hand in hands:
            flag = True
            for cond in compiled_conditions:
                if len(cond) == 1:
                    if not cond[0](hand):
                        flag = False
                        break
                elif len(cond) == 2:
                    if not cond[0](hand, cond[1]):
                        flag = False
                        break

            if flag:
                result.append(hand)
    
    return result

def testIfAllConditionsArePotBased(condition):
    if isinstance(condition[0], list):
        flag = True
        for c in condition:
            if "totalPot" not in c[0].__name__:
                flag = False
                break
        
        return flag            
    else:
        if "totalPot" in condition[0].__name__:
            return True
    
    return False

def potValuesThatSatisfyCondition(condition, pots):
    result = set([])
    
    if not isinstance(condition[0], list):
        for pot in pots:
            if condition[0](pot, condition[1]):
                result.add(pot)
    else:
        for pot in pots:
            flag = True
            for cond in condition:
                if not cond[0](pot, cond[1]):
                    flag = False
                    break
            if flag:
                result.add(pot)
    
    return result

def isTreeValid(tree, hands):
    conditions = []
    for i in range(0, len(tree)):
        if tree[i].name == "IfThenElse":
            subtree_slice = tree.searchSubtree(i)
            if tree[subtree_slice.stop-1].ret == str and tree[subtree_slice.stop-2].ret == str and tree[subtree_slice.stop-1].name == tree[subtree_slice.stop-2].name:
                return False
        
            subtree_slice = tree.searchSubtree(i+1)
            conditions.append(tree[subtree_slice.start:subtree_slice.stop])
        
    conditions = parseAllConditions(conditions)
    #print conditions
    current_hands = set(hands)
    current_pots = set(range(0, 50))
    for condition in conditions:
        temp = handsThatMeetCondition(condition, current_hands)
        
        #print "Start: " + str(len(current_hands))
        if len(temp) == len(current_hands) or len(temp) == 0:
            if not testIfAllConditionsArePotBased(condition):
                return False
            else:
                temp_pots = potValuesThatSatisfyCondition(condition, current_pots)
                if len(current_pots) == len(temp_pots) or len(temp_pots) == 0:
                    return False
            
                current_pots = current_pots.difference(temp_pots)

        #print "End: " + str(len(temp))
        current_hands = current_hands.difference(set(temp))
    
    return True

def sameCondition(cond1, cond2):
    if len(cond1) != len(cond2):
        return False

    if len(cond1) == 1:
        if cond1[0] == cond2[0]:
            return True
        return False
    
    if cond1[0] == cond2[0] and cond1[1] == cond2[1]:
        return True

    return False

def conditionToString(condition):
    result = ""
    if len(condition) == 1:
        result += condition[0].__name__ + "(ARG0)"
    else:
        if "totalPot" in condition[0].__name__:
            result += condition[0].__name__ + "(ARG1, " + str(condition[1]) + ")"
        else:
            result += condition[0].__name__ + "(ARG0, " + str(condition[1]) + ")"
    
    return result

def parseListOfTreeToString(list_tree, original):
    result = "IfThenElse("
    
    k = 1
    count = 1
    for i in range(0, len(list_tree)):
        condition = list_tree[i]
        if not isinstance(condition[0], list):
            result += conditionToString(condition)
        else:
            temp_result = ""
            temp_result += "and_(" + conditionToString(condition[-2]) + ", " + conditionToString(condition[-1]) + ")"
            for j in range(3, len(condition)+1):
                temp_result = "and_(" + conditionToString(condition[-j]) + ", " + temp_result + ")"
            result += temp_result

        subtree_slice = original.searchSubtree(k)
        k = subtree_slice.stop
        result += ", '" + original[k].name + "', "
        k += 1
        #if original[k].ret == str:
        if i == len(list_tree) - 1:
            result += ", '" + original[k].name + "'"
        else:
            result += "IfThenElse("
            k += 1
            count += 1
    
    for i in range(count):
        result += ")"
    
    return result

def makeAllVariationsOfHeuristic(ind):
    index_of_statements = []
    list_of_variations = []
    for i in range(0, len(ind)):
        if ind[i].name == "IfThenElse":
            subtree_slice = ind.searchSubtree(i)
            index_of_statements.append(subtree_slice.start)
            
    index_of_statements.append(len(ind)-1)
    
    for x in range(0, len(index_of_statements)-1):
        #list_of_statements.append(ind[index_of_statements[x]:index_of_statements[x+1]])
        temp = ind[0:index_of_statements[x]]
        temp.extend(ind[index_of_statements[x+1]:len(ind)])
        list_of_variations.append(gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(temp)), pset))
    
    return list_of_variations

def mapHandsToResults(ind, all_hands):
    result = {}
    func = toolbox.compile(ind)
    for i in range(0, 50):
        result[i] = {}
    for hand in all_hands:
        for i in range(0, 50):
            result[i][str(hand[0]) + ", " + str(hand[1])] = func(hand, i)
    
    return result

def checkIfDictionariesAreTheSame(dict1, dict2):
    for x in dict1:
        for y in dict1[x]:
            if dict1[x][y] != dict2[x][y]:
                return False

    return True

def simplifyByRemovingStatements(ind, all_hands):
    all_variations = makeAllVariationsOfHeuristic(ind)
    
    original_result_mapping = mapHandsToResults(ind, all_hands)
    for variation in all_variations:
        var_result_mapping = mapHandsToResults(variation, all_hands)
        if checkIfDictionariesAreTheSame(original_result_mapping, var_result_mapping):
            return simplifyByRemovingStatements(variation, all_hands)
    
    return ind

def returnTreeDepth(ind):
    depth = 0
    for x in ind:
        if x.name == "IfThenElse":
            depth += 1
    
    return depth

def simplifyTree(tree, hands):
    conditions = []
    for i in range(0, len(tree)):
        if tree[i].name == "IfThenElse":
            subtree_slice = tree.searchSubtree(i)
            #if tree[subtree_slice.stop-1].ret == str and tree[subtree_slice.stop-2].ret == str and tree[subtree_slice.stop-1].name == tree[subtree_slice.stop-2].name:
            #    return False
        
            subtree_slice = tree.searchSubtree(i+1)
            conditions.append(tree[subtree_slice.start:subtree_slice.stop])
        
    conditions = parseAllConditions(conditions)
    #print conditions
    current_hands = set(hands)
    result = []
    changes = False
    for condition in conditions:
        hands_dict = {}
        #print condition
        if isinstance(condition[0], list):
            #print condition
            for i in range(len(condition)):
                hands_dict[i] = set(handsThatMeetCondition(condition[i], current_hands))

            to_remove = set([])
            keys = hands_dict.keys()
            for i in range(0, len(keys)-1):
                if "totalPot" in condition[i][0].__name__:
                    continue
                for j in range(i+1, len(keys)):
                    if "totalPot" in condition[j][0].__name__:
                        continue
                    intersection = hands_dict[i].intersection(hands_dict[j])
                    length_of_intersection = len(intersection)
                    if length_of_intersection == len(hands_dict[i]):
                        to_remove.add(j)
                    if length_of_intersection == len(hands_dict[j]):
                        cond1 = condition[i]
                        cond2 = condition[j]
                        if not sameCondition(cond1, cond2):
                            to_remove.add(i)
            
            if len(to_remove) > 0:
                new_condition = []
                for i in range(0, len(condition)):
                    if i not in to_remove:
                        new_condition.append(condition[i])
                
                if len(new_condition) == 1:
                    new_condition = new_condition[0]
                if len(new_condition) == 0:
                    new_condition = condition[0]
                changes = True

                result.append(list(new_condition))
            else:
                result.append(condition)
        else:
            result.append(condition)
    
    if not changes:
        return tree
    
    return gp.PrimitiveTree.from_string(parseListOfTreeToString(result, tree), pset)

all_hands = []
for i in range(2, 15):
    for j in range(i, 15):
        all_hands.append((Card(i, "D"), Card(j, "S")))
        if i != j:
            all_hands.append((Card(i, "D"), Card(j, "D")))

def isThereCopyOfIndividual(pop, ind):
    for p in pop:
        if primitiveTreeDistance(p, ind) == 0.0:
            return True
    return False

def makePopulation(n, individual_creation_function):
    population = []
    #depth = 0
    while len(population) < n:
        ind = individual_creation_function()
        ind = simplifyTree(ind, all_hands)
        #if depth == 0:
        #    depth = returnTreeDepth(ind)
        #temp = simplifyByRemovingStatements(ind, all_hands)
        #temp_depth = returnTreeDepth(temp)
        
        if not isThereCopyOfIndividual(population, ind) and isTreeValid(ind, all_hands): #and depth == temp_depth:
            population.append(ind)
    
    return population

def individualForMutation(depth, pset, type_=None):
    result = ""
    if type_ == bool:
        result = generateBoolOp()
    if type_ == int:
        result = str(random.choice(possible_ints))
    if type_ == str:
        result = random.choice(possible_strings)

    return gp.PrimitiveTree.from_string(result, pset)

def makeMutation(indv, chosen, removed_elements):
    new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)
    #if chosen.ret != str and chosen.ret != bool and chosen.ret != int:
    while (chosen.name == new_gene[0].name):
        new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)

    index_for_mutation = indv.index(chosen)

    temp = []
    if chosen.name == "and_":
        temp_indv = gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)
        subtree_slice = temp_indv.searchSubtree(index_for_mutation)
        temp = indv[:index_for_mutation] + new_gene + indv[subtree_slice.stop:]
        for temp_element in indv[subtree_slice.start:subtree_slice.stop]:
            removed_elements.add(temp_element)
    elif isinstance(chosen, gp.Terminal):
        temp = indv[:index_for_mutation] + new_gene + indv[index_for_mutation+1:]
    elif isinstance(chosen, gp.Primitive):
        temp_indv = gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)
        subtree_slice = temp_indv.searchSubtree(index_for_mutation)
        temp = indv[:index_for_mutation] + new_gene + indv[subtree_slice.stop:]
        for temp_element in indv[subtree_slice.start:subtree_slice.stop]:
            removed_elements.add(temp_element)
    
    return list(temp)


def mutateExpressionTree(indv, mut_prob, pset):
    mutable_elements = set([])
    removed_elements = set([])
    
    for expr in indv:
        if expr.name != 'IfThenElse' and expr.name != "ARG0" and expr.name != "ARG1":
            mutable_elements.add(expr)
    
    #chosen = random.choice(mutable_elements)
    #current = gp
    count = 0
    for element in mutable_elements:
        if element not in removed_elements:
            removed_elements.add(element)
            n = random.uniform(0, 1)
            #print n
            if n <= mut_prob:
                count += 1
                chosen = element

                indv = makeMutation(indv, chosen, removed_elements)
    
    if count == 0:
        chosen = random.choice(list(mutable_elements))

        indv = makeMutation(indv, chosen, removed_elements)        
    
    return gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)

def mateIndividuals(ind1, ind2, depth, pset):
    r = random.randint(1,depth) # which level to crossover in
    n = random.randint(0,2) # which child to crossover
    
    index_of_ind1_tree = 0
    index_of_ind2_tree = 0
    
    counter = 1
    for i in range(0, len(ind1)):
        if ind1[i].arity == 3:
            if counter == r:
                index_of_ind1_tree = i
                break
            counter += 1

    counter = 1
    for i in range(0, len(ind2)):
        if ind2[i].arity == 3:
            if counter == r:
                index_of_ind2_tree = i
                break
            counter += 1
    
    subtree_slice_ind1 = ind1.searchSubtree(index_of_ind1_tree+1)
    for i in range(0, n):
        subtree_slice_ind1 = ind1.searchSubtree(subtree_slice_ind1.stop)

    subtree_slice_ind2 = ind2.searchSubtree(index_of_ind2_tree+1)
    for i in range(0, n):
        subtree_slice_ind2 = ind2.searchSubtree(subtree_slice_ind2.stop)

    new_ind1 = ind1[0:subtree_slice_ind1.start] + ind2[subtree_slice_ind2.start:subtree_slice_ind2.stop] + ind1[subtree_slice_ind1.stop:]
    new_ind2 = ind2[0:subtree_slice_ind2.start] + ind1[subtree_slice_ind1.start:subtree_slice_ind1.stop] + ind2[subtree_slice_ind2.stop:]

    return (gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(new_ind1)), pset), gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(new_ind2)), pset))

tree_depth = 5

toolbox = base.Toolbox()
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("heuristic", generateIndividual, depth=tree_depth, pset=pset)
toolbox.register("individual", tools.initIterate, creator.GameHeuristicIndividualMax, toolbox.heuristic)
#toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("population", makePopulation, individual_creation_function=toolbox.individual)
toolbox.register("select", tools.selBest)
#toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selDoubleTournament, parsimony_size=1.7, fitness_first=True, fitness_size=3)
toolbox.register("mate", mateIndividuals, depth=tree_depth, pset=pset)
#toolbox.register("mate", gp.cxOnePoint)
toolbox.register("heuristic_mut", individualForMutation, depth=tree_depth)
#toolbox.register("mutate", gp.mutUniform, expr=toolbox.heuristic_mut, pset=pset)
toolbox.register("mutate", mutateExpressionTree, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=tree_depth)) #Depth to allow ANDs
#toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=tree_depth+2)) #Depth + 2 to allow ANDs
#toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=3)) #Depth + 2 to allow ANDs

#pool = multiprocessing.Pool()
#toolbox.register("map", pool.map)


In [13]:
class HeuristicAI:
    def __init__(self):
        pass
    
    def setHeuristic(self, heuristic):
        self.heuristic = heuristic
    
    def setHand(self, hand):
        self.hand = hand
    
    def generateOffset(self):
        pass
    
    def decide(self, gamestate, available_actions):
        #hand = self.gamestate.players[self.gamestate.current_player]
        pot_size = int(float(gamestate.current_bid) / float(gamestate.big_blind))
        
        return self.heuristic(self.hand, pot_size)

In [177]:
BIG_BLIND = 20
TOTAL_PLAYER_POT = 1000
NUM_OF_PLAYERS = 8

def runSimulation(individual, n):
    number_of_players = NUM_OF_PLAYERS
    total_player_pot = TOTAL_PLAYER_POT
    small_blind = BIG_BLIND / 2

    agents = [HeuristicAI()]
    '''
    for i in range(1, number_of_players-1):
        agents.append(TierHandAI())
    agents.append(AlwaysRaiseAI())
    '''
    half_number_of_players = int(float(number_of_players) / 2.0)
    #for i in range(1, number_of_players):
    for i in range(1, half_number_of_players):
        agents.append(TierHandAI())
        agents[i].generateOffset()

    for i in range(half_number_of_players, number_of_players):
        agents.append(TopTenHandAI())

    agents[0].setHeuristic(toolbox.compile(individual))
    
    #random.seed(10)
    gh = GameHandler(number_of_players, agents, total_player_pot, small_blind)
    
    results = []
    agent_pot = []
    
    for i in range(0, n):
        result, money = gh.run()
        results.append(result)
        agent_pot.append(money)
    
    return (results, agent_pot)

def evaluate(player, n=8000):
    results, money = runSimulation(player, n)
    score = {'wins': 0.0, 'ties': 0.0, 'losses': 0.0}
    money_lost = 0.0
    
    return (sum(money),)


def evaluateMoney(player, n=8000):
    results, money = runSimulation(player, n)
    score = {'wins': 0.0, 'ties': 0.0, 'losses': 0.0}
    money_lost = 0.0
    
    return money

def evaluateVariance(player, n=8000):
    results, money = runSimulation(player, n)
    score = {'wins': 0.0, 'ties': 0.0, 'losses': 0.0}
    money_lost = 0.0

    return (money,)

#print evaluate(toolbox.individual())

In [15]:
import random, itertools
import multiprocessing as mp

def primitiveTreeDistance(ind1, ind2):
    result = 0.0
    
    min_length = min(len(ind1), len(ind2))
    factor = 1.0 / float(min_length)
    for i in range(0, min_length):
        if ind1[i].name != ind2[i].name:
            result += factor

    if len(ind1) != len(ind2):
        result += factor
        
    return result if result <= 1.0 else 1.0

def findFitness(population):
    pop = population
    
    pool = mp.Pool(mp.cpu_count())
    fitness_list = pool.map(toolbox.evaluate, pop)
    pool.terminate()
    
    for i in range(0, len(fitness_list)):
        #pop[i].fitness = creator.FitnessMax
        #pop[i].fitness.values = fitness_list[i]
        pop[i].fitness = fitness_list[i]
    
    return pop

def readjustFitnessByDistance(population):
    pop = population
    best = toolbox.select(pop, 1)[0]
    weight = 10000.0
    
    for ind in pop:
        if ind != best:
            dist = 1.0 - primitiveTreeDistance(best, ind)
            ind.fitness = (ind.fitness[0] - (weight * dist), )

def mateBest(best):
    new_indv = []
    
    for i in range(0, len(best)-1):
        for j in range(i+1, len(best)):
            children = toolbox.mate(gp.PrimitiveTree.from_string(str(best[i]), pset), gp.PrimitiveTree.from_string(str(best[j]), pset))
            children = [simplifyTree(children[0], all_hands), simplifyTree(children[1], all_hands)]
            new_indv.extend(children)

    return new_indv

def mateElite(elite, size_of_children=60):
    children = []
    
    pairings = tuple(itertools.combinations(elite, 2))
    selected = random.sample(pairings, int(float(size_of_children) / 2.0))
    
    for i in range(0, len(selected)):
        temp = toolbox.mate(gp.PrimitiveTree.from_string(str(selected[i][0]), pset), gp.PrimitiveTree.from_string(str(selected[i][1]), pset))
        temp = [simplifyTree(temp[0], all_hands), simplifyTree(temp[1], all_hands)]
        children.extend(temp)

    return children
        
def mutateChildren(children, mut_prob, pop):
    mut_children = []
    
    for child in children:
        #rand_value = random.uniform(0.0, 1.0)
        #if rand_value <= mut_prob:
        #child = toolbox.mutate(child, mut_prob)

        while isThereCopyOfIndividual(pop + mut_children, child) or not isTreeValid(child, all_hands):
            child = simplifyTree(toolbox.mutate(child, mut_prob, pset=pset), all_hands)
        
        mut_children.append(child)

    return mut_children

def createMutationIndv(indv, mut_prob, pop):
    mutation = gp.PrimitiveTree.from_string(str(indv), pset)
    mut_indv = toolbox.mutate(mutation, mut_prob, pset=pset)
    
    while isThereCopyOfIndividual(pop, mut_indv) or not isTreeValid(mut_indv, all_hands):
        mut_indv = simplifyTree(toolbox.mutate(mutation, mut_prob, pset=pset), all_hands)
    
    return mut_indv

def generation(population, mut_prob, children_mut_prob, k):
    pop = population
    pop_size = len(pop)

    elite_size = int(0.5 * pop_size)
    elite = toolbox.select(pop, elite_size)
    elite_mutations = []
    for i in range(0, int(len(elite) * 0.4)):
        elite_mutations.append(simplifyTree(createMutationIndv(elite[i], mut_prob, elite + elite_mutations), all_hands))

    children = mateElite(elite, int(0.3 * pop_size))
    children = mutateChildren(children, children_mut_prob, elite + elite_mutations)

    pop = elite + elite_mutations + children
    pop = findFitness(pop)
    
    return pop

def gen0(population):
    pop = population
    pop = findFitness(pop)
    #readjustFitnessByDistance(pop)
    
    return pop

def createLogFile(filename, start_string):
    f = open(filename, 'w')
    f.write(start_string)
    f.close()

def logToFile(filename, input_string):
    f = open(filename, 'a')
    f.write(input_string)
    f.close()

In [ ]:
import time, datetime, os

num_sims = 80000#160000 * 3
toolbox.register("evaluate", evaluate, n=num_sims)

machine_alias = "Typheus"

filename = "GP Results/gen-output-" + machine_alias + "-" + str(datetime.datetime.now()) +  ".txt"

ngen=100
k=10
mut_prob = 0.3
children_mut_prob = 0.1
pop = toolbox.population(n=200)
#hof = tools.HallOfFame(3)

start = time.time()
createLogFile(filename, "Gen 0 Start\n")

pop = gen0(pop)

logToFile(filename, "Gen 0 End\nEllapsed Time: " + str(time.time() - start) + "\n")

for i in range(0, ngen):
    start = time.time()
    logToFile(filename, "Gen " + str(i+1) + "\n")
    
    pop = generation(pop, mut_prob, children_mut_prob, k)
    
    top = toolbox.select(pop, 3)
    logToFile(filename, "Fitness: " + str(top[0].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[0]) + "\n")
    logToFile(filename, "Fitness: " + str(top[1].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[1]) + "\n")
    logToFile(filename, "Fitnees: " + str(top[2].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[2]) + "\n")
    
    logToFile(filename, "Ellapsed Time: " + str(time.time() - start) + "\n")
    
logToFile(filename, "==================================\n")
logToFile(filename, "==================================\n")
logToFile(filename, "==================================\n")
print("==================================")
print("==================================")
print("==================================")

best = toolbox.select(pop, 3)
for b in best:
    logToFile(filename, str(b) + "\n")
    logToFile(filename, str(b.fitness) + "\n")
    
csv_filename = "GP Results/mapping-" + machine_alias + ".csv.txt"
if not os.path.isfile(csv_filename):
    createLogFile(csv_filename, "Tree Depth; Population; Num Generations; Num Sims; Top1; Fitness-top1; Top2; Fitness-top2; Top3; Fitness-top3; File;\n")

logToFile(csv_filename, str(returnTreeDepth(best[0])) + "; " + str(len(pop)) + "; " + str(ngen) + "; " + str(num_sims) + "; " + str(best[0]) + "; " + str(best[0].fitness) + "; "  + str(best[1]) + "; " + str(best[1].fitness) + "; " + str(best[2]) + "; " + str(best[2].fitness) + "; " + str(filename) + ";\n")


In [111]:
def generateAllHeuristicTrees(depth=1, ands=0):
    main = "IfThenElse("
    one_parameter_functions = ["isSameSuit", "hasDoubles"]
    two_parameter_functions = ["highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]
    two_parameter_functions_arg1 = ["totalPotGE", "totalPotLE"]
    int_parameters = range(2, 15)
    int_parameters_arg1 = range(1, 30)
    string_parameters = ["'raise'", "'check'", "'fold'"]
    special_operations = ["and_"]

    first_param = []
    
    for func in one_parameter_functions:
        result = func + '(ARG0)'
        first_param.append(result)
    
    for func in two_parameter_functions:
        for param_int in int_parameters:
            result = func + '(ARG0,'
            result += str(param_int) + ')'
            first_param.append(result)

    for func in two_parameter_functions_arg1:
        for param_int in int_parameters_arg1:
            result = func + '(ARG1,'
            result += str(param_int) + ')'
            first_param.append(result)

    '''
    temp = []
    for func1 in first_param:
        for func2 in first_param:
            if func1.split("(")[0] != func2.split("(")[0]:
                temp.append("and_(" + func1 + "," + func2 + ")")
    
    first_param.extend(temp)
    '''
                
    second_param = list(string_parameters)
    
    third_param = []
    if depth == 1:
        third_param = list(string_parameters)
    else:
        third_param = generateAllHeuristicTrees(depth-1)
    
    trees = []
    if ands == 0:
        for x in first_param:
            for y in second_param:
                for z in third_param:
                    trees.append(main + x + ',' + y + ',' + z + ')')
    elif ands == 1:
        for i in range(0, len(first_param)-1):
            for j in range(i+1, len(first_param)):
                for y in second_param:
                    for z in third_param:
                        trees.append(main + 'and_(' + first_param[i] + ',' + first_param[j] + '),' + y + ',' + z + ')')
    
    return trees

def addStringToHeuristic(heuristic_str, new_statement_str):
    comma_pos = heuristic_str.rfind(',')
    comma_pos2 = new_statement_str.rfind(',')

    return heuristic_str[0:comma_pos+2] + new_statement_str[0:comma_pos2] + heuristic_str[comma_pos:] + ")"

def generateListPrimitiveTreesFromListStrings(list_strings, default=None):
    result = []
    for tree_string in list_strings:
        if default == None:
            result.append(gp.PrimitiveTree.from_string(tree_string, pset))
        else:
            result.append(gp.PrimitiveTree.from_string(addStringToHeuristic(default, tree_string), pset))
    
    return result

In [ ]:
## Exhaustive search
num_sims = 160000 * 3

filename = "Exhaustive Search/exs-output-" + machine_alias + "-" + str(datetime.datetime.now()) +  ".txt"
createLogFile(filename, "Exhaustive Search\nNumber of Simulations: " + str(num_sims) + "\n")

toolbox.register("evaluate", evaluate, n=num_sims)

pop = generateListPrimitiveTreesFromListStrings(generateAllHeuristicTrees(1))
pop = findFitness(pop)

for x in toolbox.select(pop, 20):
    logToFile(filename, str(x) + "\n")
    logToFile(filename, str(x.fitness) + "\n")
    print x
    print "Fitness: " + str(x.fitness)

In [139]:
## Exhaustive search (with removal of broken individuals)
import datetime

num_sims = 160000 * 3
machine_alias = "Typheus"

filename = "Exhaustive Search/exs-output-" + machine_alias + "-" + str(datetime.datetime.now()) +  ".txt"
createLogFile(filename, "Exhaustive Search\nNumber of Simulations: " + str(num_sims) + "\n")

temp_pop = generateListPrimitiveTreesFromListStrings(generateAllHeuristicTrees(1, 0), "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', 'raise'))))")
print len(temp_pop)
to_remove = []
for p in temp_pop:
    if not isTreeValid(p, all_hands):
        to_remove.append(p)
    else:
        temp_p = simplifyByRemovingStatements(simplifyTree(p, all_hands), all_hands)
        if findComplexity(p) != findComplexity(temp_p):
            to_remove.append(p)

#for p in to_remove:
#    pop.remove(p)
pop = []
for p in temp_pop:
    if not isThereCopyOfIndividual(pop, p) and p not in to_remove:
        pop.append(p)

print len(pop)

toolbox.register("evaluate", evaluate, n=num_sims)

pop = findFitness(pop)

for x in toolbox.select(pop, 20):
    logToFile(filename, str(x) + ";" + str(x.fitness) + ";\n")
    print x
    print "Fitness: " + str(x.fitness)

1008
131
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 17), 'check', 'raise')))))
Fitness: (38615445,)
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 29), 'check', 'raise')))))
Fitness: (38596743,)
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotLE(ARG1, 1), 'check', 'raise')))))
Fitness: (38581793,)
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0

In [125]:
testing = ["IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))"] * 10

testing_tree = [gp.PrimitiveTree.from_string(x, pset) for x in testing]

toolbox.register("evaluate", evaluate, n=160000 * 3)
testing_tree = findFitness(testing_tree)

for x in toolbox.select(testing_tree, 10):
    print x
    print "Fitness: " + str((float(x.fitness[0]) / (160000.0 * 3.0)) - 80.0)

IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.631314583333
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.607772916667
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.496225
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.462772916667
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.448466666667
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.409414583333
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.3593875
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
Fitness: 0.354422916667
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(total

In [318]:
testing = [
    "IfThenElse(totalPotLE(ARG1, 4), 'raise', 'check')",
    "IfThenElse(totalPotGE(ARG1, 4), 'check', 'raise')",
    "IfThenElse(lowestCardLE(ARG0, 8), 'fold', 'raise')",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', 'raise')",
    "IfThenElse(lowestCardGE(ARG0, 10), 'raise', 'fold')",
    "IfThenElse(lowestCardLE(ARG0, 7), 'fold', 'raise')",
    "IfThenElse(lowestCardGE(ARG0, 9), 'raise', 'fold')",
    "IfThenElse(lowestCardGE(ARG0, 11), 'raise', 'fold')",
    "IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')",
    "IfThenElse(lowestCardGE(ARG0, 8), 'raise', 'fold')",
    "IfThenElse(totalPotGE(ARG1, 5), 'check', 'raise')",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', 'fold')",
]

testing_tree = [gp.PrimitiveTree.from_string(x, pset) for x in testing]

toolbox.register("evaluate", evaluate, n=80000)
testing_tree = findFitness(testing_tree)

for x in toolbox.select(testing_tree, 10):
    print x
    print "Fitness: " + str(x.fitness)

IfThenElse(lowestCardGE(ARG0, 12), 'raise', 'fold')
Fitness: (1603726143,)
IfThenElse(lowestCardGE(ARG0, 11), 'raise', 'fold')
Fitness: (1601136246,)
IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')
Fitness: (1599738400,)
IfThenElse(lowestCardGE(ARG0, 10), 'raise', 'fold')
Fitness: (1598929937,)
IfThenElse(lowestCardLE(ARG0, 9), 'fold', 'raise')
Fitness: (1598086089,)
IfThenElse(lowestCardLE(ARG0, 8), 'fold', 'raise')
Fitness: (1592459988,)
IfThenElse(lowestCardGE(ARG0, 9), 'raise', 'fold')
Fitness: (1591243369,)
IfThenElse(lowestCardGE(ARG0, 8), 'raise', 'fold')
Fitness: (1585495827,)
IfThenElse(lowestCardLE(ARG0, 7), 'fold', 'raise')
Fitness: (1582340928,)
IfThenElse(totalPotGE(ARG1, 5), 'check', 'raise')
Fitness: (1577437042,)


In [207]:
def calculateVariance(values, window_size):
    std_dev = 0.0
    mean = 0.0
    mean = float(sum(values[0:window_size]))
    mean = mean / float(window_size)
    std_dev = sum([(float(x) - mean) ** 2 for x in values[0:window_size]])
    std_dev = std_dev / float(window_size)
    
    return std_dev ** 2

In [205]:
testing_var = [
    others[0],
    others[9],
    others[73],
    others[3],
    others[71],
    others[5],
    others[32],
    others[26],
    others[10],
    others[39],
    others[42],
]

testing_tree_var = [gp.PrimitiveTree.from_string(x, pset) for x in testing_var]

toolbox.register("evaluate", evaluateVariance, n=8000000)
testing_tree_var = findFitness(testing_tree_var)

In [210]:
tests = [800, 8000, 80000, 160000, 160000 * 2, 160000 * 3, 160000 * 4, 160000 * 5]
for k in range(0, len(testing_var)):
    values = [x.fitness[0] for x in testing_tree_var]
    subject = k
    avg_values = []
    for x in tests:
        temp = []
        size = 8000000.0 / float(x)
        for i in range(0, int(size)):
            temp.append(float(sum(values[subject][i*x:((i+1)*x)])) / float(x))
        avg_values.append(temp)
    #print avg_values
    print "Subject " + str(k)
    for i in range(0, len(tests)):
        print str(tests[i]) + " : " + str(calculateVariance(avg_values[i], len(avg_values[i])))
    print "=================="
    print "\n"

Subject 0
800 : 0.00682536143524
8000 : 6.62733430421e-05
80000 : 7.74350333047e-07
160000 : 1.21488506507e-07
320000 : 2.2578661131e-08
480000 : 7.33389848921e-09
640000 : 8.11540438236e-10
800000 : 2.50315865334e-09


Subject 1
800 : 0.00210339182323
8000 : 2.10728896503e-05
80000 : 1.60423905403e-07
160000 : 4.09390728478e-08
320000 : 1.61169967653e-08
480000 : 9.66668881874e-09
640000 : 1.03789348891e-09
800000 : 1.34980463759e-09


Subject 2
800 : 0.0151318101424
8000 : 0.00014690750883
80000 : 1.42666619761e-06
160000 : 2.95384352318e-07
320000 : 8.99319035109e-08
480000 : 2.38965949332e-08
640000 : 1.37313750815e-08
800000 : 9.28303947411e-09


Subject 3
800 : 0.00620134197218
8000 : 5.61243720739e-05
80000 : 7.12377617721e-07
160000 : 1.4768628464e-07
320000 : 3.24423160578e-08
480000 : 4.70774676749e-09
640000 : 5.67457239368e-09
800000 : 1.98838919157e-09


Subject 4
800 : 0.0138156632232
8000 : 0.000140156284361
80000 : 8.66757636766e-07
160000 : 7.47970741022e-08
320000 : 1

In [184]:
others = [
    #Exhaustive
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', 'raise')",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', IfThenElse(totalPotGE(ARG1, 12), 'check', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', 'raise'))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(lowestCardGE(ARG0, 14), totalPotGE(ARG1, 11)), 'check', 'raise'))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(totalPotGE(ARG1, 15), 'check', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))))"
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', 'raise'))))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 17), 'check', 'raise')))))",
    "IfThenElse(and_(lowestCardLE(ARG0, 11), totalPotGE(ARG1, 1)), 'fold', 'raise')",
    #Typheus
    "IfThenElse(lowestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardLE(ARG0, 4), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 8), 'fold', IfThenElse(totalPotGE(ARG1, 11), 'check', 'fold')))))",
    "IfThenElse(and_(lowestCardLE(ARG0, 2), highestCardGE(ARG0, 7)), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(totalPotGE(ARG1, 24), 'raise', IfThenElse(totalPotGE(ARG1, 8), 'check', 'fold')))))",
    "IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(totalPotLE(ARG1, 7), 'raise', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 16), 'raise', IfThenElse(totalPotLE(ARG1, 14), 'raise', 'check')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 4), 'check', 'fold')))))",
    "IfThenElse(and_(totalPotGE(ARG1, 1), lowestCardGE(ARG0, 12)), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 3), 'fold', IfThenElse(totalPotLE(ARG1, 4), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise')))))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(highestCardLE(ARG0, 11), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 3), 'raise', IfThenElse(totalPotLE(ARG1, 5), 'check', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardLE(ARG0, 6), 'fold', IfThenElse(highestCardGE(ARG0, 10), 'fold', IfThenElse(lowestCardLE(ARG0, 7), 'check', 'raise')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(lowestCardGE(ARG0, 7), 'fold', IfThenElse(totalPotLE(ARG1, 4), 'fold', IfThenElse(highestCardGE(ARG0, 12), 'check', IfThenElse(highestCardLE(ARG0, 10), 'check', 'fold')))))",
    "IfThenElse(and_(highestCardGE(ARG0, 14), isSameSuit(ARG0)), 'fold', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 13), 'fold', IfThenElse(isSameSuit(ARG0), 'check', 'fold')))))",
    "IfThenElse(highestCardLE(ARG0, 3), 'fold', IfThenElse(lowestCardGE(ARG0, 11), 'raise', IfThenElse(totalPotLE(ARG1, 9), 'fold', IfThenElse(highestCardGE(ARG0, 6), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', 'check')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardLE(ARG0, 4), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 6), 'fold', IfThenElse(totalPotGE(ARG1, 27), 'check', 'fold')))))",
    "IfThenElse(and_(hasDoubles(ARG0), highestCardLE(ARG0, 14)), 'raise', IfThenElse(highestCardLE(ARG0, 13), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 6), 'fold', IfThenElse(totalPotLE(ARG1, 24), 'check', 'fold')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardGE(ARG0, 4), 'fold', IfThenElse(totalPotGE(ARG1, 9), 'check', 'raise')))))",
    "IfThenElse(lowestCardGE(ARG0, 14), 'raise', IfThenElse(lowestCardLE(ARG0, 6), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'fold', IfThenElse(totalPotLE(ARG1, 27), 'fold', 'raise')))))",
    "IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotLE(ARG1, 12), 'check', IfThenElse(highestCardLE(ARG0, 10), 'check', IfThenElse(isSameSuit(ARG0), 'check', 'raise')))))",
    #----
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 26), 'check', IfThenElse(totalPotGE(ARG1, 8), 'check', IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')))))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(and_(totalPotGE(ARG1, 5), totalPotLE(ARG1, 9)), 'check', IfThenElse(totalPotGE(ARG1, 12), 'check', IfThenElse(totalPotLE(ARG1, 7), 'raise', 'check')))))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 12), 'check', IfThenElse(totalPotGE(ARG1, 11), 'check', IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')))))",
    #Abraxas
    "IfThenElse(and_(hasDoubles(ARG0), highestCardGE(ARG0, 9)), 'raise', IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 23), 'fold', 'raise'))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardGE(ARG0, 14), 'check', IfThenElse(totalPotLE(ARG1, 16), 'fold', IfThenElse(highestCardLE(ARG0, 6), 'check', 'raise'))))",
    "IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(and_(highestCardLE(ARG0, 12), totalPotGE(ARG1, 20)), 'raise', IfThenElse(isSameSuit(ARG0), 'raise', 'check'))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotLE(ARG1, 24), 'fold', IfThenElse(hasDoubles(ARG0), 'check', IfThenElse(highestCardLE(ARG0, 13), 'fold', 'raise'))))",
    "IfThenElse(and_(lowestCardGE(ARG0, 8), and_(hasDoubles(ARG0), totalPotGE(ARG1, 1))), 'raise', IfThenElse(totalPotLE(ARG1, 13), 'fold', IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(highestCardGE(ARG0, 8), 'raise', 'check'))))",
    "IfThenElse(highestCardLE(ARG0, 7), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(hasDoubles(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 15), 'raise', 'fold'))))",
    "IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(hasDoubles(ARG0), 'check', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(and_(lowestCardLE(ARG0, 11), lowestCardGE(ARG0, 2)), 'fold', 'check'))))",
    "IfThenElse(lowestCardGE(ARG0, 14), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 9), 'check', IfThenElse(totalPotGE(ARG1, 7), 'raise', 'fold'))))",
    "IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 13), 'raise', 'fold'))))",
    "IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(highestCardGE(ARG0, 14), 'fold', IfThenElse(totalPotLE(ARG1, 29), 'fold', IfThenElse(lowestCardLE(ARG0, 2), 'check', 'raise'))))",
    "IfThenElse(lowestCardLE(ARG0, 2), 'fold', IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 16), 'raise', IfThenElse(totalPotGE(ARG1, 1), 'raise', 'check')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(and_(highestCardLE(ARG0, 7), lowestCardGE(ARG0, 6)), 'fold', IfThenElse(lowestCardLE(ARG0, 4), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotGE(ARG1, 1), 'fold', 'raise')))))",
    "IfThenElse(highestCardLE(ARG0, 4), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(lowestCardGE(ARG0, 10), 'check', IfThenElse(and_(totalPotGE(ARG1, 23), totalPotGE(ARG1, 5)), 'fold', 'check')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(lowestCardLE(ARG0, 5), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 7), 'raise', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotLE(ARG1, 20), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 5), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', 'fold')))))",
    "IfThenElse(and_(hasDoubles(ARG0), lowestCardLE(ARG0, 6)), 'check', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(and_(totalPotGE(ARG1, 12), totalPotLE(ARG1, 24)), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 29), 'check', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 3), 'check', IfThenElse(totalPotLE(ARG1, 26), 'fold', IfThenElse(highestCardLE(ARG0, 11), 'fold', 'raise')))))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(lowestCardGE(ARG0, 11), 'check', IfThenElse(and_(highestCardGE(ARG0, 14), totalPotGE(ARG1, 12)), 'check', 'fold')))))",
    "IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'check', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(totalPotLE(ARG1, 24), 'fold', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(hasDoubles(ARG0), 'fold', 'check')))))",
    #---
    "IfThenElse(and_(highestCardGE(ARG0, 7), hasDoubles(ARG0)), 'raise', IfThenElse(lowestCardGE(ARG0, 11), 'raise', 'fold'))",
    "IfThenElse(and_(highestCardGE(ARG0, 7), hasDoubles(ARG0)), 'raise', IfThenElse(lowestCardGE(ARG0, 13), 'raise', 'fold'))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', 'check'))",
    #Cetus
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', 'fold')))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardGE(ARG0, 4), 'fold', IfThenElse(totalPotLE(ARG1, 24), 'fold', 'raise')))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardLE(ARG0, 3), 'check', IfThenElse(totalPotGE(ARG1, 19), 'check', 'fold')))",
    "IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(and_(highestCardGE(ARG0, 14), isSameSuit(ARG0)), 'fold', IfThenElse(totalPotGE(ARG1, 1), 'raise', 'fold')))",
    "IfThenElse(highestCardLE(ARG0, 2), 'fold', IfThenElse(highestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', 'fold')))",
    "IfThenElse(and_(lowestCardGE(ARG0, 8), hasDoubles(ARG0)), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardLE(ARG0, 12), 'fold', 'check')))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(totalPotGE(ARG1, 27), 'fold', 'check')))",
    "IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(lowestCardLE(ARG0, 12), 'check', IfThenElse(isSameSuit(ARG0), 'fold', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 12), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 29), 'raise', 'check')))",
    "IfThenElse(and_(hasDoubles(ARG0), lowestCardLE(ARG0, 14)), 'raise', IfThenElse(totalPotLE(ARG1, 23), 'fold', IfThenElse(highestCardLE(ARG0, 9), 'check', 'fold')))",
    "IfThenElse(lowestCardGE(ARG0, 14), 'raise', IfThenElse(highestCardLE(ARG0, 12), 'fold', IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', 'raise')))))",
    "IfThenElse(highestCardLE(ARG0, 2), 'fold', IfThenElse(highestCardLE(ARG0, 6), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(totalPotLE(ARG1, 11), 'fold', 'raise')))))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 13), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 9), 'fold', IfThenElse(lowestCardLE(ARG0, 6), 'check', 'raise')))))",
    "IfThenElse(and_(lowestCardGE(ARG0, 12), highestCardGE(ARG0, 12)), 'raise', IfThenElse(lowestCardLE(ARG0, 5), 'fold', IfThenElse(totalPotLE(ARG1, 4), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(hasDoubles(ARG0), 'check', 'raise')))))",
    "IfThenElse(and_(hasDoubles(ARG0), highestCardGE(ARG0, 7)), 'raise', IfThenElse(and_(isSameSuit(ARG0), totalPotGE(ARG1, 11)), 'raise', IfThenElse(hasDoubles(ARG0), 'check', IfThenElse(lowestCardGE(ARG0, 9), 'fold', IfThenElse(highestCardGE(ARG0, 4), 'fold', 'check')))))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(and_(highestCardGE(ARG0, 9), totalPotGE(ARG1, 20)), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(totalPotLE(ARG1, 23), 'fold', 'check')))))",
    "IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(highestCardLE(ARG0, 2), 'check', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 8), 'fold', IfThenElse(highestCardGE(ARG0, 11), 'fold', 'check')))))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 19), 'check', 'fold')))))",
    "IfThenElse(highestCardLE(ARG0, 5), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 7), 'fold', IfThenElse(highestCardGE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 28), 'raise', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(lowestCardLE(ARG0, 7), 'fold', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(lowestCardGE(ARG0, 12), 'check', IfThenElse(highestCardGE(ARG0, 13), 'fold', 'check')))))",
    #---
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 7), 'check', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotGE(ARG1, 11), 'check', 'raise')))",
    "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 14), 'check', 'raise')))",
    #Hemera
    "IfThenElse(lowestCardGE(ARG0, 11), 'raise', IfThenElse(hasDoubles(ARG0), 'check', 'fold'))",
    "IfThenElse(and_(lowestCardLE(ARG0, 9), and_(hasDoubles(ARG0), lowestCardGE(ARG0, 2))), 'check', IfThenElse(hasDoubles(ARG0), 'raise', 'fold'))",
    "IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(hasDoubles(ARG0), 'check', 'fold'))",
    "IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', 'raise'))",
    "IfThenElse(and_(highestCardGE(ARG0, 10), hasDoubles(ARG0)), 'raise', IfThenElse(highestCardLE(ARG0, 13), 'fold', 'check'))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(and_(highestCardGE(ARG0, 2), lowestCardLE(ARG0, 11)), 'fold', 'raise'))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', 'check'))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotGE(ARG1, 27), 'check', 'fold'))",
    "IfThenElse(lowestCardGE(ARG0, 10), 'raise', IfThenElse(totalPotLE(ARG1, 11), 'fold', 'check'))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 3), 'check', 'fold'))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 8), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(totalPotGE(ARG1, 11), 'check', 'raise')))))",
    "IfThenElse(and_(highestCardGE(ARG0, 6), hasDoubles(ARG0)), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'fold', IfThenElse(highestCardLE(ARG0, 6), 'fold', IfThenElse(totalPotLE(ARG1, 12), 'fold', IfThenElse(totalPotLE(ARG1, 16), 'check', 'raise')))))",
    "IfThenElse(highestCardLE(ARG0, 5), 'fold', IfThenElse(highestCardLE(ARG0, 7), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(totalPotGE(ARG1, 27), 'raise', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardGE(ARG0, 5), 'fold', IfThenElse(highestCardLE(ARG0, 3), 'check', IfThenElse(totalPotLE(ARG1, 18), 'fold', IfThenElse(totalPotLE(ARG1, 23), 'fold', 'raise')))))",
    "IfThenElse(lowestCardGE(ARG0, 14), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotLE(ARG1, 22), 'fold', IfThenElse(highestCardGE(ARG0, 4), 'raise', 'check')))))",
    "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 11), 'fold', IfThenElse(isSameSuit(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 4), 'check', IfThenElse(highestCardGE(ARG0, 12), 'check', 'fold')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(highestCardLE(ARG0, 5), 'fold', IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 13), 'check', 'fold')))))",
    "IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardLE(ARG0, 4), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(totalPotGE(ARG1, 2), 'check', 'fold')))))",
    "IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(totalPotLE(ARG1, 11), 'fold', IfThenElse(and_(lowestCardLE(ARG0, 10), lowestCardGE(ARG0, 7)), 'raise', IfThenElse(highestCardLE(ARG0, 11), 'fold', IfThenElse(lowestCardGE(ARG0, 11), 'check', 'fold')))))",
    "IfThenElse(and_(highestCardLE(ARG0, 3), hasDoubles(ARG0)), 'fold', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(highestCardLE(ARG0, 10), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 3), 'fold', 'check')))))",
    #---
    "IfThenElse(highestCardLE(ARG0, 6), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 13), 'fold', IfThenElse(totalPotGE(ARG1, 7), 'check', 'raise'))))",
    "IfThenElse(and_(hasDoubles(ARG0), highestCardGE(ARG0, 7)), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(totalPotLE(ARG1, 10), 'raise', IfThenElse(isSameSuit(ARG0), 'raise', 'check'))))",
    "IfThenElse(and_(hasDoubles(ARG0), highestCardGE(ARG0, 10)), 'raise', IfThenElse(lowestCardLE(ARG0, 11), 'fold', IfThenElse(totalPotLE(ARG1, 10), 'check', IfThenElse(and_(highestCardLE(ARG0, 13), isSameSuit(ARG0)), 'fold', 'check'))))",
]

others_primitive_tree = []
to_remove = []
for i in range(0, len(others)):
    heuristic = simplifyTree(simplifyByRemovingStatements(gp.PrimitiveTree.from_string(others[i], pset), all_hands), all_hands)
    if not isThereCopyOfIndividual(others_primitive_tree, heuristic):
        others_primitive_tree.append(heuristic)
        others[i] = str(others_primitive_tree[-1])
    else:
        to_remove.append(others[i])

for item in to_remove:
    others.remove(item)

toolbox.register("evaluate", evaluate, n=160000 * 5)

evaluated_individuals = findFitness(others_primitive_tree)

In [203]:
temp_others = [
    others[0],
    others[9],
    others[73],
    others[3],
    others[71],
    others[5],
    others[32],
    others[26],
    others[10],
    others[39],
    others[42],
]

temp_others_primitive_tree = []
for x in temp_others:
    temp_others_primitive_tree.append(gp.PrimitiveTree.from_string(x, pset))

toolbox.register("evaluate", evaluateMoney, n=160000 * 5)

temp_evaluated_individuals = findFitness(temp_others_primitive_tree)

In [204]:
filename = "VarTesting/vart-output-" + machine_alias + "-" + str(datetime.datetime.now()) +  ".csv"
createLogFile(filename, "")

for x in temp_evaluated_individuals:
    result = ""
    for y in x.fitness:
        result += str(y) + ','
    #print result[0:20]
    logToFile(filename, result + "\n")


In [107]:
import math

temp = ["IfThenElse(and_(lowestCardLE(ARG0, 11), totalPotGE(ARG1, 1)), 'fold', IfThenElse(and_(lowestCardLE(ARG0, 11), totalPotGE(ARG1, 1)), 'fold', 'raise'))"]
#print len(temp)

temp_primitive_tree = []
for x in temp:
    temp_primitive_tree.append(gp.PrimitiveTree.from_string(x, pset))


In [187]:
for ind in evaluated_individuals:
    print ind
    #print (float(ind.fitness[0]))
    print (float(ind.fitness[0]) / (160000.0 * 5)) - 1000.0

IfThenElse(lowestCardLE(ARG0, 10), 'fold', 'raise')
2.364
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 19), 'check', 'raise'))
2.364
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(totalPotGE(ARG1, 12), 'check', 'raise'))
2.364
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', 'raise'))
2.2872375
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(lowestCardGE(ARG0, 14), totalPotGE(ARG1, 11)), 'check', 'raise'))
2.364
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(totalPotGE(ARG1, 15), 'check', 'raise')))
2.2872375
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highestCardLE(ARG0, 12), 'check', 'raise')))
2.0648125
IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 12)), 'fold', IfThenElse(highe

In [18]:
def findComplexity(ind):
    complexity = 0
    
    for expr in ind:
        if expr.name != "IfThenElse" and expr.name != "and_":
            if isinstance(expr, gp.Terminal):
                if expr.name == "raise" or expr.name == "fold" or expr.name == "check":
                    complexity += 1
            else:
                complexity += 1
    
    return complexity

In [188]:
for i in range(0, len(evaluated_individuals)):
    print str([(float(evaluated_individuals[i].fitness[0]) / (160000.0 * 5)) - 1000.0, findComplexity(evaluated_individuals[i])]) + ","

#for i in range(0, len(evaluated_individuals)):
#    print str([(float(evaluated_individuals[i].fitness[0]) / (160000.0 * 3)) - 1000.0, findComplexity(evaluated_individuals[i])]) + ","
#    print evaluated_individuals[i]
#    print i

[2.3640000000000327, 3],
[2.3640000000000327, 5],
[2.3640000000000327, 5],
[2.2872374999999465, 6],
[2.3640000000000327, 6],
[2.2872374999999465, 8],
[2.0648125000000164, 8],
[1.8498749999999973, 10],
[1.8384750000000167, 12],
[1.3053374999999505, 4],
[3.0886500000000296, 11],
[1.3053374999999505, 12],
[1.3053374999999505, 11],
[0.9526499999999487, 11],
[1.3053374999999505, 12],
[2.3640000000000327, 9],
[1.8868062499999496, 11],
[1.3053374999999505, 11],
[0.7160999999999831, 12],
[2.3640000000000327, 11],
[1.2002999999999702, 11],
[2.5030124999999543, 11],
[2.504175000000032, 11],
[1.3053374999999505, 9],
[2.7978562499999953, 11],
[3.7522500000000036, 7],
[3.7522500000000036, 10],
[1.5077625000000126, 10],
[2.415668750000009, 9],
[1.5216500000000224, 10],
[1.3053374999999505, 9],
[0.8389875000000302, 9],
[1.6012375000000247, 9],
[1.3735812500000293, 9],
[1.556137499999977, 7],
[0.7122000000000526, 9],
[0.5561249999999518, 9],
[1.3053374999999505, 7],
[1.0703624999999874, 12],
[2.872475

In [215]:
def runSimulationForTrees(n):
    number_of_players = NUM_OF_PLAYERS
    total_player_pot = TOTAL_PLAYER_POT
    small_blind = BIG_BLIND / 2

    heuristics = [
        others[29],
        others[14],
        others[14],
        others[14],
        others[14],
        others[14],
        others[14],
        others[14],
        #others[7],
        #others[10],
        #others[12],
        #others[8],
        #others[14],
        #others[15],
        #others[16],
    ]
    
    agents = []
    for i in range(0, number_of_players):
        agents.append(HeuristicAI())
        agents[i].setHeuristic(toolbox.compile(gp.PrimitiveTree.from_string(heuristics[i], pset)))
    #agents.append(AlwaysRaiseAI())

    #agents[0].setHeuristic(toolbox.compile(individual))
    #agents[1].setHeuristic(toolbox.compile(individual))
    
    gh = GameHandler(number_of_players, agents, total_player_pot, small_blind)
    
    results = []
    agent_pot = []
    
    for i in range(0, n):
        result, money = gh.runGetAllPlayerPots()
        results.append(result)
        agent_pot.append(money)
    
    return results, agent_pot

In [216]:
results, agent_pot = runSimulationForTrees(160000 * 3)

In [197]:
def calculateStats(results, agent_pot):
    wins = {}
    money = {}
    for x in results:
        for num in x:
            if num not in wins:
                wins[num] = 1
                money[num] = 0
            else:
                wins[num] += 1

    for agent_money in agent_pot:
        for i in range(0, len(agent_money)):
            money[i] += agent_money[i]
    
    return wins, money

In [214]:
wins, money = calculateStats(results, agent_pot)
print "Wins: "
for x in wins:
    print str(x+1) + " : " + str(wins[x])
print "+++++==========+++++"
print "Money: "
for x in money:
    print "[" + str(x+1) + ", " + str(float(money[x]) / (160000.0 * 3) - 1000.0) + "],"


Wins: 
1 : 81707
2 : 56939
3 : 57066
4 : 56824
5 : 56968
6 : 56901
7 : 57024
8 : 56887
+++++==========+++++
Money: 
[1, -1.81747291667],
[2, 0.263989583333],
[3, 0.205654166667],
[4, 0.24375625],
[5, 0.480197916667],
[6, 0.173772916667],
[7, 0.2636],
[8, 0.18600625],


In [73]:
def makeAllVariationsOfHeuristic(ind):
    index_of_statements = []
    list_of_variations = []
    for i in range(0, len(ind)):
        if ind[i].name == "IfThenElse":
            subtree_slice = ind.searchSubtree(i)
            index_of_statements.append(subtree_slice.start)
            
    index_of_statements.append(len(ind)-1)
    
    for x in range(0, len(index_of_statements)-1):
        #list_of_statements.append(ind[index_of_statements[x]:index_of_statements[x+1]])
        temp = ind[0:index_of_statements[x]]
        temp.extend(ind[index_of_statements[x+1]:len(ind)])
        list_of_variations.append(gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(temp)), pset))
    
    return list_of_variations

def mapHandsToResults(ind, all_hands):
    result = {}
    func = toolbox.compile(ind)
    for hand in all_hands:
        result[str(hand[0]) + ", " + str(hand[1])] = func(hand, 0)
    
    return result

def checkIfDictionariesAreTheSame(dict1, dict2):
    for x in dict1:
        if dict1[x] != dict2[x]:
            return False

    return True

def simplifyByRemovingStatements(ind, all_hands):
    all_variations = makeAllVariationsOfHeuristic(ind)
    
    original_result_mapping = mapHandsToResults(ind, all_hands)
    for variation in all_variations:
        var_result_mapping = mapHandsToResults(variation, all_hands)
        if checkIfDictionariesAreTheSame(original_result_mapping, var_result_mapping):
            return simplifyByRemovingStatements(variation, all_hands)
    
    return ind

def returnTreeDepth(ind):
    depth = 0
    for x in ind:
        if x.name == "IfThenElse":
            depth += 1
    
    return depth

temp_ind = gp.PrimitiveTree.from_string("IfThenElse(and_(highestCardLE(ARG0, 8), hasDoubles(ARG0)), 'check', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 7), 'fold', IfThenElse(lowestCardGE(ARG0, 8), 'fold', 'raise')))))", pset)
print temp_ind
aux = simplifyByRemovingStatements(temp_ind, all_hands)
print returnTreeDepth(temp_ind)
print returnTreeDepth(aux)



IfThenElse(and_(highestCardLE(ARG0, 8), hasDoubles(ARG0)), 'check', IfThenElse(lowestCardGE(ARG0, 12), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 7), 'fold', IfThenElse(lowestCardGE(ARG0, 8), 'fold', 'raise')))))
5
4


In [27]:
import time, datetime, os

machine_alias = "Typheus" 

def returnListOfPossibleValues(function_string):
    possible_strings = [
                "raise",
                "check",
                "fold"
               ]

    possible_ints = range(2, 15)
    possible_pot_ints = range(1, 30)
    
    if 'totalPot' in function_string:
        return possible_pot_ints
    elif function_string == "action":
        return possible_strings

    return possible_ints

def extractListOfFieldsToChange(ind):
    result = []
    last_func = None
    for i in range(0, len(ind)):
        element = ind[i]
        if isinstance(element, gp.Terminal) and 'ARG' not in element.name:
            if element.ret == str:
                result.append((element, i , "action"))
            else:
                result.append((element, i, last_func))
        elif element.name[:3] != 'ARG':
            last_func = element.name
            
    return result

def createNewIndividual(ind, value, index):
    return gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(ind[:index] + [gp.Terminal(value, False, type(value))] + ind[index+1:])), pset)

def checkIfAnyDictionariesAreTheSame(dict1, list_of_dicts):
    if len(list_of_dicts) == 0:
        return False
    
    for x in list_of_dicts:
        if checkIfDictionariesAreTheSame(dict1, x):
            return True
    
    return False

def indexOfDifference(ind1, ind2):
    result = -1
    for i in range(0, len(ind1)):
        if ind1[i].name != ind2[i].name:
            result = i
            break
    
    return result

NUMBER_OF_GAMES_PLAYED = 80000#160000 * 3
toolbox.register("evaluate", evaluate, n=NUMBER_OF_GAMES_PLAYED)#n=160000 * 3)

def axis_aligned_search(tree_size, ind=None):
    filename = "AAS Results/aas-output-" + machine_alias + "-" + str(datetime.datetime.now()) + ".txt"
    isRandom = True if ind == None else False
    
    if ind == None:
        while True:
            ind = generateIndividual(tree_size, pset)
            temp = simplifyByRemovingStatements(ind, all_hands)
            temp_depth = returnTreeDepth(temp)

            if isTreeValid(ind, all_hands) and temp_depth == tree_size:
                break

        ind = simplifyTree(ind, all_hands)
    else:
        ind = gp.PrimitiveTree.from_string(ind, pset)
    list_of_elements_to_change = extractListOfFieldsToChange(ind)
    
    start = time.time()
    createLogFile(filename, "Default Ind:\n" + str(ind) + "\n\n")

    count = len(list_of_elements_to_change)
    current_ind = ind
    #print ind
    while count > 0:
        variants_population = [current_ind]
        variants_map_results = [mapHandsToResults(current_ind, all_hands)]
        for i in range(0, len(list_of_elements_to_change)):
            terminal_to_change_tuple = list_of_elements_to_change[i]
            possible_values = returnListOfPossibleValues(terminal_to_change_tuple[2])
            for x in possible_values:
                new_ind = createNewIndividual(current_ind, x, terminal_to_change_tuple[1])
                new_ind_hand_map = mapHandsToResults(new_ind, all_hands)

                if isTreeValid(new_ind, all_hands) and not checkIfAnyDictionariesAreTheSame(new_ind_hand_map, variants_map_results):
                    variants_population.append(new_ind)
                    variants_map_results.append(new_ind_hand_map)
        
        variants_population = findFitness(variants_population)
        new_current_ind = toolbox.select(variants_population, 1)[0]
        
        index = indexOfDifference(current_ind, new_current_ind)
        if index == -1:
            break
            #return current_ind
        else:
            for x in list_of_elements_to_change:
                if x[1] == index:
                    temp = x
                    break
                    
            list_of_elements_to_change.remove(temp)
        
        current_ind = new_current_ind

        logToFile(filename, "Count " + str(count) + "\n" + str(current_ind) + " \nEllapsed Time: " + str(time.time() - start) + "\n\n")
        start = time.time()
        
        count = count - 1
        
    logToFile(filename, "=====================\n")
    logToFile(filename, "Result: " + str(current_ind) + " \nEllapsed Time: " + str(time.time() - start) + "\n")
    logToFile(filename, "Fitness: " + str(current_ind.fitness[0]) + "\n")
    
    csv_filename = "AAS Results/mapping-" + machine_alias + ".csv.txt"
    if not os.path.isfile(csv_filename):
        createLogFile(csv_filename, "Starting Tree; isRandom; EndTree; num of games; Fitness; File;\n")

    logToFile(csv_filename, str(ind) + "; " + str(isRandom) + "; " + str(current_ind) + "; " + str(NUMBER_OF_GAMES_PLAYED) + "; " + str(current_ind.fitness[0]) + "; " + str(filename) + ";\n")
    
    return current_ind


In [ ]:
#result = axis_aligned_search(4, "IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 3)), 'fold', IfThenElse(lowestCardLE(ARG0, 8), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 5), 'check', 'raise'))))")
for i in range(50):
    print axis_aligned_search(5)

In [23]:
print result
print result.fitness
print float(result.fitness[0]) / (160000.0 * 3) - 1000.0

IfThenElse(and_(isSameSuit(ARG0), highestCardLE(ARG0, 3)), 'check', IfThenElse(lowestCardLE(ARG0, 10), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 24), 'check', 'raise'))))
(481395062,)
2.90637916667


In [30]:
temp_heuristic = "IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 26), 'check', IfThenElse(totalPotGE(ARG1, 8), 'check', IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')))))"
temp_tree = gp.PrimitiveTree.from_string(temp_heuristic, pset)
print simplifyByRemovingStatements(temp_tree, all_hands)

IfThenElse(lowestCardLE(ARG0, 9), 'fold', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotLE(ARG1, 3), 'raise', 'check')))


In [71]:
def outputReadableHeuristic(heuristic_tree):
    result = ""
    default = "IF "
    and_count = 0
    for expr in heuristic_tree:
        if expr.name == "IfThenElse":
            result += default
            default = "ELSE IF "
            
        elif "LE" == expr.name[-2:]:
            result += expr.name[:-2] + " <= "
        elif "GE" == expr.name[-2:]:
            result += expr.name[:-2] + " >= "
        elif isinstance(expr, gp.Terminal):
            if "ARG" in expr.name:
                continue
            if expr.ret == str:
                if expr == heuristic_tree[-1]:
                    result += "ELSE " + expr.name.upper()
                else:
                    result += "THEN " + expr.name.upper() + "\n"
            else:
                result += expr.name + " "
                if and_count > 0:
                    result += " AND "
                    and_count -= 1
        elif expr.name != "and_":
            result += expr.name + " "
            if and_count > 0:
                result += " AND "
                and_count -= 1
        else:
            and_count += 1
    
    return result

In [79]:
temp = [
        gp.PrimitiveTree.from_string(others[0], pset),
        gp.PrimitiveTree.from_string(others[29], pset),
        gp.PrimitiveTree.from_string(others[30], pset),
        gp.PrimitiveTree.from_string(others[102], pset),
        gp.PrimitiveTree.from_string(others[72], pset),
        gp.PrimitiveTree.from_string(others[59], pset),
        gp.PrimitiveTree.from_string(others[40], pset),
        gp.PrimitiveTree.from_string(others[86], pset),
        gp.PrimitiveTree.from_string(others[15], pset),
        gp.PrimitiveTree.from_string(others[22], pset),
]

complexities = [3, 5, 6, 7, 8, 9, 10, 11, 12, 13]

for i in range(0, len(complexities)):
    print "Complexity " + str(complexities[i]) + ":"
    print outputReadableHeuristic(temp[i])
    print "\n"

Complexity 3:
IF lowestCard >= 12 THEN RAISE
ELSE FOLD


Complexity 5:
IF hasDoubles  AND lowestCard <= 7 THEN CHECK
ELSE IF hasDoubles THEN RAISE
ELSE IF lowestCard <= 9 THEN FOLD
ELSE IF totalPot <= 6 THEN RAISE
ELSE CHECK


Complexity 6:
IF lowestCard <= 9 THEN FOLD
ELSE IF hasDoubles THEN RAISE
ELSE IF totalPot <= 6  AND totalPot <= 3 THEN RAISE
ELSE CHECK


Complexity 7:
IF lowestCard <= 9 THEN FOLD
ELSE IF hasDoubles THEN RAISE
ELSE IF totalPot <= 4 THEN RAISE
ELSE CHECK


Complexity 8:
IF lowestCard <= 8 THEN FOLD
ELSE IF hasDoubles THEN RAISE
ELSE IF totalPot <= 9 THEN RAISE
ELSE CHECK


Complexity 9:
IF totalPot >= 22  AND isSameSuit  AND lowestCard >= 2  AND totalPot >= 28 THEN FOLD
ELSE IF lowestCard >= 13 THEN RAISE
ELSE FOLD


Complexity 10:
IF lowestCard <= 4  AND totalPot >= 10 THEN CHECK
ELSE IF lowestCard <= 9 THEN FOLD
ELSE IF hasDoubles THEN RAISE
ELSE IF totalPot >= 5 THEN CHECK
ELSE RAISE


Complexity 11:
IF hasDoubles  AND lowestCard >= 7 THEN RAISE
ELSE IF totalP